# Load Packages

In [576]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from category_encoders import BinaryEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import IsolationForest

import category_encoders as ce
import pickle
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif
import klib as kl

In [577]:
import warnings
warnings.filterwarnings('ignore')

## Load Data

In [578]:
df_train = pd.read_csv('data/mimic_train.csv')
df_test = pd.read_csv('data/mimic_test_death.csv')
diagnosis = pd.read_csv('data/MIMIC_metadata_diagnose.csv')
metadata_df = pd.read_csv('data/MIMIC_diagnoses.csv')

print(len(df_train['hadm_id'].unique()))

target = 'los'

id_cols = ['subject_id', 'hadm_id', 'icustay_id']

final_output_df = df_test.copy()

19749


In [579]:
short_diagnose_dict = dict(zip(diagnosis.ICD9_CODE, diagnosis.SHORT_DIAGNOSE))
long_diagnose_dict = dict(zip(diagnosis.ICD9_CODE, diagnosis.LONG_DIAGNOSE))

## Create LOS features for each ICD9 code

get all codes from the dataset and left join them with the ICD9 codes from the main dataset table

In [580]:
comorbidities = metadata_df.groupby('HADM_ID').agg(
    UNIQUE_DIAGNOSIS_COUNT=('ICD9_CODE', 'nunique'),  # Count of unique ICD9 codes
    DIAGNOSIS_COUNT=('ICD9_CODE', 'count'),
    ICD9_CODE_LIST=('ICD9_CODE', lambda x: list(x.unique()))  # List of unique ICD9 codes
).reset_index()

df_train = pd.merge(df_train, comorbidities, left_on='hadm_id', right_on='HADM_ID',how='left').drop(columns='HADM_ID')

df_test = pd.merge(df_test, comorbidities, left_on='hadm_id', right_on='HADM_ID',how='left').drop(columns='HADM_ID')

target_encoder_cols = []

for i in range(1, 40):
    df_train[f'code{i}'] = df_train['ICD9_CODE_LIST'].apply(lambda x: x[i-1] if len(x) > i-1 else None)
    df_test[f'code{i}'] = df_test['ICD9_CODE_LIST'].apply(lambda x: x[i-1] if len(x) > i-1 else None)
    target_encoder_cols.append(f'code{i}')


metadata_df.columns = ['subject_id', 'hadm_id', 'SEQ_NUM', 'ICD9_CODE']

print(target_encoder_cols)

['code1', 'code2', 'code3', 'code4', 'code5', 'code6', 'code7', 'code8', 'code9', 'code10', 'code11', 'code12', 'code13', 'code14', 'code15', 'code16', 'code17', 'code18', 'code19', 'code20', 'code21', 'code22', 'code23', 'code24', 'code25', 'code26', 'code27', 'code28', 'code29', 'code30', 'code31', 'code32', 'code33', 'code34', 'code35', 'code36', 'code37', 'code38', 'code39']


We now get the LOS for each ICD9 code so we can sum it up later

In [581]:
X_train_merge = df_train[["subject_id", "hadm_id", "LOS"]]
meta_data_merge = pd.merge(metadata_df, X_train_merge, how = "left", on = ["subject_id", "hadm_id"])

ICD9_mean = meta_data_merge.groupby('ICD9_CODE').agg({'LOS' : 'mean'}).rename(columns = {'LOS': 'mean'})
meta_data_merge = pd.merge(meta_data_merge, ICD9_mean, how = "left", on = "ICD9_CODE")

meta_data_merge_wide = pd.DataFrame(meta_data_merge.pivot_table(index = ["subject_id", "hadm_id"], columns = "SEQ_NUM", values = "mean"))

Collapse the data to get the LOS metrics for all patients

In [582]:

name_covariates = meta_data_merge_wide.columns.tolist()
coveriates_of_interest = [col for col in name_covariates if isinstance(col, float) and 1.0 <= col <= 39.0]

# Select the columns and calculate the mean, ignoring NaN values with skipna=True
meta_data_merge_wide['ICD9_mean'] = meta_data_merge_wide[coveriates_of_interest].mean(axis=1, skipna=True)
meta_data_merge_wide['ICD9_max'] = meta_data_merge_wide[coveriates_of_interest].max(axis=1, skipna=True)
meta_data_merge_wide['ICD9_sum'] = meta_data_merge_wide[coveriates_of_interest].sum(axis=1, skipna=True)
meta_data_merge_wide = meta_data_merge_wide.drop(coveriates_of_interest, axis = 1)

df_train = pd.merge(df_train, meta_data_merge_wide, how = "left", on = ["subject_id", "hadm_id"])
df_test = pd.merge(df_test, meta_data_merge_wide, how = "left", on = ["subject_id", "hadm_id"])
df_train = df_train.drop("ICD9_diagnosis", axis = 1)
df_test = df_test.drop("ICD9_diagnosis", axis = 1)


## Feature Extraction

cleaning up vars to get age for each icu stay

In [583]:
df_train['ADMITTIME'] = pd.to_datetime(df_train['ADMITTIME'])
df_train['DOB'] = pd.to_datetime(df_train['DOB'])
df_test['ADMITTIME'] = pd.to_datetime(df_test['ADMITTIME'])
df_test['DOB'] = pd.to_datetime(df_test['DOB'])

df_train['ADMITTIME'] += pd.to_timedelta(df_train['Diff'], unit = 'D')
df_train['DOB'] += pd.to_timedelta(df_train['Diff'], unit = 'D')
df_test['ADMITTIME'] += pd.to_timedelta(df_test['Diff'], unit = 'D')
df_test['DOB'] += pd.to_timedelta(df_test['Diff'], unit = 'D')

df_train.loc[df_train['DOB'].dt.year < 1900, 'DOB'] = np.nan
df_test.loc[df_test['DOB'].dt.year < 1900, 'DOB'] = np.nan

df_train['AGE'] = round((df_train['ADMITTIME'] - df_train['DOB']).dt.days / 365.25)
df_test['AGE'] = round((df_test['ADMITTIME'] - df_test['DOB']).dt.days / 365.25)

df_train.loc[df_train['AGE'].isna(), 'AGE'] = 93.0
df_test.loc[df_train['AGE'].isna(), 'AGE'] = 93.0


Now we get more detailed information on the date of admission 

In [584]:


df_train['ADMIT_HOUR'] = df_train['ADMITTIME'].dt.hour
df_test['ADMIT_HOUR'] = df_test['ADMITTIME'].dt.hour

df_train['ADMIT_MONTH'] = df_train['ADMITTIME'].dt.hour
df_test['ADMIT_MONTH'] = df_test['ADMITTIME'].dt.hour

df_train['ADMIT_YEAR'] = df_train['ADMITTIME'].dt.hour
df_test['ADMIT_YEAR'] = df_test['ADMITTIME'].dt.hour

# turn hour categorical by shift with 3 shifts (7-15, 15-23, 23-7)
df_train['ADMIT_SHIFT'] = pd.cut(df_train['ADMIT_HOUR'], bins=[0, 7, 15, 23], labels=['night', 'morning', 'evening'])
df_test['ADMIT_SHIFT'] = pd.cut(df_test['ADMIT_HOUR'], bins=[0, 7, 15, 23], labels=['night', 'morning', 'evening'])
df_train['ADMIT_SHIFT'] = df_train['ADMIT_SHIFT'].astype(str)
df_test['ADMIT_SHIFT'] = df_test['ADMIT_SHIFT'].astype(str)

# turn month categorical by season
df_train['ADMIT_SEASON'] = pd.cut(df_train['ADMIT_MONTH'], bins=[0, 3, 6, 9, 12], labels=['winter', 'spring', 'summer', 'fall'])
df_test['ADMIT_SEASON'] = pd.cut(df_test['ADMIT_MONTH'], bins=[0, 3, 6, 9, 12], labels=['winter', 'spring', 'summer', 'fall'])
df_train['ADMIT_SEASON'] = df_train['ADMIT_SEASON'].astype(str)
df_test['ADMIT_SEASON'] = df_test['ADMIT_SEASON'].astype(str)


Now we get number of icu stays for each patient

In [585]:


df_train_count = df_train[['subject_id', 'ADMITTIME', 'icustay_id']].copy()
df_test_count = df_test[['subject_id', 'ADMITTIME', 'icustay_id']].copy()

df_train_count = df_train_count.sort_values(by = ['subject_id', 'ADMITTIME'])
df_test_count = df_test_count.sort_values(by = ['subject_id', 'ADMITTIME'])

df_train_count['previous_icu_stays'] = df_train_count.groupby('subject_id').cumcount()
df_test_count['previous_icu_stays'] = df_test_count.groupby('subject_id').cumcount()

df_train = pd.merge(df_train, df_train_count[['icustay_id', 'previous_icu_stays']], on = 'icustay_id', how = 'left')
df_test = pd.merge(df_test, df_test_count[['icustay_id', 'previous_icu_stays']], on = 'icustay_id', how = 'left')


df_train['prev_icu_visit'] = np.where(df_train['previous_icu_stays'] == 0, 0, 1)
df_test['prev_icu_visit'] = np.where(df_test['previous_icu_stays'] == 0, 0, 1)

df_train.drop('previous_icu_stays', axis = 1, inplace = True)
df_test.drop('previous_icu_stays', axis = 1, inplace = True)


Bin the age variable

In [586]:
# bin age variable into 0–2, 3–5, 6–13, 14–18, 19–33, 34–48, 49–64, 65–78, and 79–98

df_train['AGE_bin'] = pd.cut(df_train['AGE'], bins = [0, 2, 5, 13, 18, 33, 48, 64, 78, 98], labels = ['0-2', '3-5', '6-13', '14-18', '19-33', '34-48', '49-64', '65-78', '79-98'])
df_test['AGE_bin'] = pd.cut(df_test['AGE'], bins = [0, 2, 5, 13, 18, 33, 48, 64, 78, 98], labels = ['0-2', '3-5', '6-13', '14-18', '19-33', '34-48', '49-64', '65-78', '79-98'])

df_train['AGE_bin'] = df_train['AGE_bin'].astype(str)
df_test['AGE_bin'] = df_test['AGE_bin'].astype(str)


## Data Cleanup

Drop the columns that are not useful anymore.

In [587]:
cols_to_drop = ['subject_id',
                'hadm_id',
                'icustay_id',
                'DOD',
                'DISCHTIME',
                'DEATHTIME',
                'Diff',
                'ADMITTIME',
                'DOB',
                'ICD9_CODE_LIST',
                'DIAGNOSIS',
                'ICD9_diagnosis'
                ]

for col in cols_to_drop:
    if col in df_train.columns:
        df_train.drop(col, axis = 1, inplace = True)
    else:
        pass
    
    if col in df_test.columns:
        df_test.drop(col, axis = 1, inplace = True)
    else:
        pass

Get columns by type

In [588]:
def get_num_cat_cols():
    n_cols = df_test.select_dtypes(include = ['number']).columns.tolist()
    c_cols = df_test.select_dtypes(include = ['object']).columns.tolist()
    
    return n_cols, c_cols

num_cols, cat_cols = get_num_cat_cols()

print(num_cols)
print(cat_cols)

['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min', 'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean', 'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min', 'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean', 'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max', 'Glucose_Mean', 'UNIQUE_DIAGNOSIS_COUNT', 'DIAGNOSIS_COUNT', 'ICD9_mean', 'ICD9_max', 'ICD9_sum', 'AGE', 'ADMIT_HOUR', 'ADMIT_MONTH', 'ADMIT_YEAR', 'prev_icu_visit']
['GENDER', 'ADMISSION_TYPE', 'INSURANCE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'FIRST_CAREUNIT', 'code1', 'code2', 'code3', 'code4', 'code5', 'code6', 'code7', 'code8', 'code9', 'code10', 'code11', 'code12', 'code13', 'code14', 'code15', 'code16', 'code17', 'code18', 'code19', 'code20', 'code21', 'code22', 'code23', 'code24', 'code25', 'code26', 'code27', 'code28', 'code29', 'code30', 'code31', 'code32', 'code33', 'code34', 'code35', 'code36', 'code37', 'code38', 'code39', 'ADMIT_SHIFT', 

## Consolidate Categories

In [589]:
num_cols, cat_cols = get_num_cat_cols()

cat_cols = [c for c in cat_cols if c not in target_encoder_cols]

print(num_cols)
print(cat_cols)

['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min', 'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean', 'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min', 'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean', 'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max', 'Glucose_Mean', 'UNIQUE_DIAGNOSIS_COUNT', 'DIAGNOSIS_COUNT', 'ICD9_mean', 'ICD9_max', 'ICD9_sum', 'AGE', 'ADMIT_HOUR', 'ADMIT_MONTH', 'ADMIT_YEAR', 'prev_icu_visit']
['GENDER', 'ADMISSION_TYPE', 'INSURANCE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'FIRST_CAREUNIT', 'ADMIT_SHIFT', 'ADMIT_SEASON', 'AGE_bin']


Using the preprocessing from the model solution 

In [590]:
df_train['ETHNICITY'] = df_train['ETHNICITY'].replace(['ASIAN', 'ASIAN - CHINESE', 'ASIAN - ASIAN INDIAN', 'ASIAN - VIETNAMESE', 'ASIAN - FILIPINO', 'ASIAN - CAMBODIAN',
                                                     'ASIAN - JAPANESE', 'ASIAN - THAI', 'ASIAN - OTHER', 'ASIAN - KOREAN'
                                                     ], 'ASIAN')

df_train['ETHNICITY'] = df_train['ETHNICITY'].replace(['HISPANIC 0R LATINO', 'HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC/LATINO - DOMINICAN',
                                                     'HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - CUBAN', 'HISPANIC/LATINO - SALVADORAN',
                                                     'HISPANIC/LATINO - MEXICAN', 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)', 'HISPANIC/LATINO - COLOMBIAN',
                                                     'HISPANIC/LATINO - HONDURAN', 'SOUTH AMERICAN'
                                                     ], 'HISPANIC OR LATINO')

df_train['ETHNICITY'] = df_train['ETHNICITY'].replace(['WHITE', 'WHITE - RUSSIAN', 'WHITE - OTHER EUROPEAN', 'WHITE - EASTERN EUROPEAN',
                                                     'WHITE - BRAZILIAN'
                                                     ], 'WHITE')

df_train['ETHNICITY'] = df_train['ETHNICITY'].replace(['BLACK/AFRICAN', 'BLACK/AFRICAN AMERICAN', 'BLACK/CAPE VERDEAN', 'BLACK/HAITIAN'
                                                     ], 'BLACK')

df_train['ETHNICITY'] = df_train['ETHNICITY'].replace(['UNABLE TO OBTAIN', 'UNKNOWN/NOT SPECIFIED', 'PATIENT DECLINED TO ANSWER'
                                                     ], 'UNKNOWN')

df_train['ETHNICITY'] = df_train['ETHNICITY'].replace(['AMERICAN INDIAN/ALASKA NATIVE', 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE',
                                                     'CARIBBEAN ISLAND', 'MIDDLE EASTERN', 'OTHER', 'PORTUGUESE', 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
                                                     'MULTI RACE ethnicity'
                                                     ], 'OTHER')

df_test['ETHNICITY'] = df_test['ETHNICITY'].replace(['ASIAN', 'ASIAN - CHINESE', 'ASIAN - ASIAN INDIAN', 'ASIAN - VIETNAMESE', 'ASIAN - FILIPINO', 'ASIAN - CAMBODIAN',
                                                     'ASIAN - JAPANESE', 'ASIAN - THAI', 'ASIAN - OTHER', 'ASIAN - KOREAN'
                                                     ], 'ASIAN')

df_test['ETHNICITY'] = df_test['ETHNICITY'].replace(['HISPANIC 0R LATINO', 'HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC/LATINO - DOMINICAN',
                                                     'HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - CUBAN', 'HISPANIC/LATINO - SALVADORAN',
                                                     'HISPANIC/LATINO - MEXICAN', 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)', 'HISPANIC/LATINO - COLOMBIAN',
                                                     'HISPANIC/LATINO - HONDURAN', 'SOUTH AMERICAN'
                                                     ], 'HISPANIC OR LATINO')

df_test['ETHNICITY'] = df_test['ETHNICITY'].replace(['WHITE', 'WHITE - RUSSIAN', 'WHITE - OTHER EUROPEAN', 'WHITE - EASTERN EUROPEAN',
                                                     'WHITE - BRAZILIAN'
                                                     ], 'WHITE')

df_test['ETHNICITY'] = df_test['ETHNICITY'].replace(['BLACK/AFRICAN', 'BLACK/AFRICAN AMERICAN', 'BLACK/CAPE VERDEAN', 'BLACK/HAITIAN'
                                                     ], 'BLACK')

df_test['ETHNICITY'] = df_test['ETHNICITY'].replace(['UNABLE TO OBTAIN', 'UNKNOWN/NOT SPECIFIED', 'PATIENT DECLINED TO ANSWER'
                                                     ], 'UNKNOWN')

df_test['ETHNICITY'] = df_test['ETHNICITY'].replace(['AMERICAN INDIAN/ALASKA NATIVE', 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE',
                                                     'CARIBBEAN ISLAND', 'MIDDLE EASTERN', 'OTHER', 'PORTUGUESE', 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
                                                     'MULTI RACE ethnicity'
                                                     ], 'OTHER')

In [591]:
religion_other = ['HEBREW', 'UNITARIAN-UNIVERSALIST', 'HINDU', 'GREEK ORTHODOX',"JEHOVAH'S WITNESS", "BUDDHIST", 'MUSLIM', 'OTHER', 'CHRISTIAN SCIENTIST', 'EPISCOPALIAN', 'ROMANIAN EAST. ORTH', '7TH DAY ADVENTIST']
df_train['RELIGION'] = df_train['RELIGION'].replace(religion_other, 'OTHER')
df_test['RELIGION'] = df_test['RELIGION'].replace(religion_other, 'OTHER')

In [592]:

df_train.loc[df_train['MARITAL_STATUS'].isna(), 'MARITAL_STATUS'] = 'UNKNOWN'
df_train.loc[df_train['MARITAL_STATUS'] == 'UNKNOWN (DEFAULT)', 'MARITAL_STATUS'] = 'UNKNOWN'
df_test.loc[df_test['MARITAL_STATUS'].isna(), 'MARITAL_STATUS'] = 'UNKNOWN'
df_test.loc[df_test['MARITAL_STATUS'] == 'UNKNOWN (DEFAULT)', 'MARITAL_STATUS'] = 'UNKNOWN'


## Create Vital Sign Features

Here we snake case the columns 

In [593]:
df_test = kl.clean_column_names(df_test)
df_train = kl.clean_column_names(df_train)

Here we generate some vital sign features - we could potentially reduce dimensionality here, but will save the features for now. Most of these features came from chatGPT. 

In [594]:

def feature_engineering(df):
    df['shock_index'] = df['heart_rate_mean'] / df['sys_bp_mean']
    df['mean_arterial_pressure'] = df['dias_bp_mean'] + (df['sys_bp_mean'] - df['dias_bp_mean']) / df.shape[0]
    df['temp_variability'] = df['temp_c_max'] - df['temp_c_min']
    df['oxygen_saturation_variability'] = df['sp_o2_max'] - df['sp_o2_min']
    df['glucose_variability'] = df['glucose_max'] - df['glucose_min']
    df['cardiovasular_risk'] = df[['heart_rate_mean', 'sys_bp_mean', 'dias_bp_mean', 'mean_bp_mean']].mean(axis=1)
    df['pulse_pressure'] = df['sys_bp_mean'] - df['dias_bp_mean']
    df['oxygenation_index'] = df['resp_rate_mean'] / df['sp_o2_mean'] 
    df['thermal_stress_index'] = df['temp_c_mean'] * df['heart_rate_mean']
    df['fever_indicator'] = df['temp_c_mean'] > 37.5
    df['fever_indicator'] = df['fever_indicator'].astype(int)

    return df

df_train = feature_engineering(df_train)
df_test = feature_engineering(df_test)

## Text Data
Later we could potentially use this in a BERT model. 

First we map the text data to each 'code' column using a dictionary defined at the beginning of the notebook

In [595]:
code_cols = [col for col in df_train.columns if 'code' in col]

df_train[code_cols] = df_train[code_cols].fillna('')
df_test[code_cols] = df_test[code_cols].fillna('')

short_cols = []
long_cols = []
short_diagnose_dict[''] = ''
long_diagnose_dict[''] = ''

for col in code_cols:
    short_col = col + '_short_text'
    df_train[short_col] = df_train[col].astype(str).map(short_diagnose_dict)
    df_test[short_col] = df_test[col].astype(str).map(short_diagnose_dict)
    short_cols.append(short_col)

    long_col = col + '_long_text'
    df_train[long_col] = df_train[col].astype(str).map(long_diagnose_dict)
    df_test[long_col] = df_test[col].astype(str).map(long_diagnose_dict)
    long_cols.append(long_col)

df_train.columns

Index(['hospital_expire_flag', 'heart_rate_min', 'heart_rate_max',
       'heart_rate_mean', 'sys_bp_min', 'sys_bp_max', 'sys_bp_mean',
       'dias_bp_min', 'dias_bp_max', 'dias_bp_mean',
       ...
       'code35_short_text', 'code35_long_text', 'code36_short_text',
       'code36_long_text', 'code37_short_text', 'code37_long_text',
       'code38_short_text', 'code38_long_text', 'code39_short_text',
       'code39_long_text'],
      dtype='object', length=173)

Now we create a concatenated text field 

In [596]:
all_cols = long_cols + short_cols
df_train['all_text'] = df_train[all_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis = 1)
df_test['all_text'] = df_test[all_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis = 1)



Now we drop all of the text and code columns

In [597]:
df_test.drop(columns = all_cols, axis = 1, inplace = True)
df_train.drop(columns = all_cols, axis = 1, inplace = True)

Now we get numerical columns for imputation

In [598]:
# get all numerical columns
num_cols, cat_cols = get_num_cat_cols()
cat_cols = [col for col in cat_cols if col not in ['all_text']]
print(num_cols)
print(cat_cols)

['heart_rate_min', 'heart_rate_max', 'heart_rate_mean', 'sys_bp_min', 'sys_bp_max', 'sys_bp_mean', 'dias_bp_min', 'dias_bp_max', 'dias_bp_mean', 'mean_bp_min', 'mean_bp_max', 'mean_bp_mean', 'resp_rate_min', 'resp_rate_max', 'resp_rate_mean', 'temp_c_min', 'temp_c_max', 'temp_c_mean', 'sp_o2_min', 'sp_o2_max', 'sp_o2_mean', 'glucose_min', 'glucose_max', 'glucose_mean', 'unique_diagnosis_count', 'diagnosis_count', 'icd9_mean', 'icd9_max', 'icd9_sum', 'age', 'admit_hour', 'admit_month', 'admit_year', 'prev_icu_visit', 'shock_index', 'mean_arterial_pressure', 'temp_variability', 'oxygen_saturation_variability', 'glucose_variability', 'cardiovasular_risk', 'pulse_pressure', 'oxygenation_index', 'thermal_stress_index', 'fever_indicator']
['gender', 'admission_type', 'insurance', 'religion', 'marital_status', 'ethnicity', 'first_careunit', 'code1', 'code2', 'code3', 'code4', 'code5', 'code6', 'code7', 'code8', 'code9', 'code10', 'code11', 'code12', 'code13', 'code14', 'code15', 'code16', 'co

## Impute Missing Values

Standard KNN imputer. I found 10 neighbors worked well in a previous notebook on a different similar dimension sized dataset. 

In [599]:
# import knn imputer
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10)

df_train_imputed = imputer.fit_transform(df_train[num_cols])

df_train_imputed = pd.DataFrame(df_train_imputed, columns = num_cols, index = df_train.index)

df_test_imputed = imputer.transform(df_test[num_cols])

df_test_imputed = pd.DataFrame(df_test_imputed, columns = num_cols, index = df_test.index)

df_train[num_cols] = df_train_imputed

df_test[num_cols] = df_test_imputed

## Prepare columns for encoding

In [600]:
one_hot_encode_cols = ['gender','marital_status', 'ethnicity', 'age_bin']
binary_encode_cols = ['religion']
target_encode_cols = [col for col in cat_cols if col not in one_hot_encode_cols and not col in binary_encode_cols]



#### Binary encode columns

Binary encoding is a good choice for columns with low cardinality. Compared to one hot encoding, it reduces the number of columns created.

In [601]:
binary_encoder = ce.BinaryEncoder(cols = binary_encode_cols, return_df = True)

df_train_binary = df_train[binary_encode_cols].copy()
df_test_binary = df_test[binary_encode_cols].copy()

binary_tr = binary_encoder.fit(df_train[binary_encode_cols])

df_train_binary_encd = binary_tr.transform(df_train_binary)
df_test_binary_encd = binary_tr.transform(df_test_binary)

df_train = pd.concat([df_train.drop(columns = binary_encode_cols, axis = 1,), df_train_binary_encd], axis = 1)
df_test = pd.concat([df_test.drop(columns = binary_encode_cols, axis = 1,), df_test_binary_encd], axis = 1)

#### Target encode columns

Target encoding encodes the column based on the target metric, in this case LOS. 

In [602]:
# target encode cat_cols 
target_encoder = ce.TargetEncoder(cols = target_encode_cols, return_df = True)

df_train_target = df_train[target_encode_cols].copy()
df_test_target = df_test[target_encode_cols].copy()

target_tr = target_encoder.fit(df_train_target, df_train['los'])

df_train_target_encd = target_tr.transform(df_train_target)
df_test_target_encd = target_tr.transform(df_test_target)

df_train = pd.concat([df_train.drop(columns = target_encode_cols, axis = 1,), df_train_target_encd], axis = 1)
df_test = pd.concat([df_test.drop(columns = target_encode_cols, axis = 1,), df_test_target_encd], axis = 1)


#### One hot encode columns

One hot encoding creates binary columns for each category type. Compared to binary encoding, it is better for columns with high cardinality.

In [603]:
print(df_train.shape)

# one hot encode cat_cols
one_hot_encoder = ce.OneHotEncoder(cols = one_hot_encode_cols, return_df = True)

df_train_one_hot = df_train[one_hot_encode_cols].copy()
df_test_one_hot = df_test[one_hot_encode_cols].copy()

one_hot_tr = one_hot_encoder.fit(df_train_one_hot)

df_train_one_hot_encd = one_hot_tr.transform(df_train_one_hot)
df_test_one_hot_encd = one_hot_tr.transform(df_test_one_hot)

df_train = pd.concat([df_train.drop(columns = one_hot_encode_cols, axis = 1,), df_train_one_hot_encd], axis = 1)
df_test = pd.concat([df_test.drop(columns = one_hot_encode_cols, axis = 1,), df_test_one_hot_encd], axis = 1)

print(df_train.shape)

(20885, 98)
(20885, 116)


## Scale Columns

First getting columns by skewness to determine which to robust scale and which to log transform

In [604]:
num_cols, cat_cols = get_num_cat_cols()

In [605]:
df_train = df_train.drop(columns = ['hospital_expire_flag'], axis = 1)

# get columns with skew greater than 0.5
skew_cols = df_train[num_cols].apply(lambda x: x.skew()).sort_values(ascending = False)
skew_cols = skew_cols[skew_cols > 0.5].index.tolist()

# get columns with 0,1 only
binary_cols = df_train[cat_cols].apply(lambda x: x.nunique()).sort_values()
binary_cols = binary_cols[binary_cols == 2].index.tolist()

# get columns not in binary and skew columns
other_cols = [col for col in df_train.columns if col not in binary_cols]
other_cols = [col for col in other_cols if col not in skew_cols]
other_cols.remove('los')
other_cols.remove('all_text')

#### Log transform columns

As we saw in class, we can first log scale then use a regular scaler. So here we log transform the columns with skewness > 1. 

In [606]:

cols_to_log = [col for col in num_cols if abs(df_train[col].skew()) > 1]

#log transform skewed columns
for col in cols_to_log:
    df_train[col] = np.log1p(df_train[col])
    df_test[col] = np.log1p(df_test[col])

#### Robust Scale Columns

We found columns with skew > .5 so we will robust scale those columns. Robust scaling scales the data based on the median and IQR.


In [607]:
# robust scale skew_cols
robust_scaler = RobustScaler()

df_train_robust = df_train[skew_cols].copy()
df_test_robust = df_test[skew_cols].copy()

robust_tr = robust_scaler.fit(df_train_robust)

df_train_robust_scaled = robust_tr.transform(df_train_robust)
df_test_robust_scaled = robust_tr.transform(df_test_robust)

df_train_robust_scaled = pd.DataFrame(df_train_robust_scaled, columns = df_train_robust.columns, index = df_train_robust.index)
df_test_robust_scaled = pd.DataFrame(df_test_robust_scaled, columns = df_test_robust.columns, index = df_test_robust.index)

df_train = pd.concat([df_train.drop(columns = skew_cols, axis = 1,), df_train_robust_scaled], axis = 1)
df_test = pd.concat([df_test.drop(columns = skew_cols, axis = 1,), df_test_robust_scaled], axis = 1)


#### Standard Scale Columns

We found columns with skew < .5 so we will standard scale those columns. Standard scaling scales the data based on the mean and standard deviation.

In [608]:

# standard scale non_skew_cols and dummy_cols
standard_scaler = StandardScaler()

df_train_standard = df_train[list(set(other_cols + binary_cols))].copy()
df_test_standard = df_test[list(set(other_cols + binary_cols))].copy()

standard_tr = standard_scaler.fit(df_train_standard)

df_train_standard_scaled = standard_tr.transform(df_train_standard)
df_test_standard_scaled = standard_tr.transform(df_test_standard)

df_train_standard_scaled = pd.DataFrame(df_train_standard_scaled, columns = df_train_standard.columns, index = df_train_standard.index)
df_test_standard_scaled = pd.DataFrame(df_test_standard_scaled, columns = df_test_standard.columns, index = df_test_standard.index)

df_train = pd.concat([df_train.drop(columns = list(set(other_cols + binary_cols)), axis = 1,), df_train_standard_scaled], axis = 1)
df_test = pd.concat([df_test.drop(columns = list(set(other_cols + binary_cols)), axis = 1,), df_test_standard_scaled], axis = 1)


Finally, we save the data to a csv file for use in the models.

In [609]:
# save to csv
df_train.to_csv('data/train_cleaned_2.csv', index = False)
df_test.to_csv('data/test_cleaned_2.csv', index = False)
